<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [35]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [36]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99]):
      self.period = period
      self.increased = increased
      self.traded = traded

      self.price_dic = {}

      self.start, self.today = self.get_date()
      self.df = self.get_stock_df()
      self.filtered_df = self.get_filtered_df()
      self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    return self.filtered_df.sort_values('Sharpe', ascending=False)

  def plot_df(self, start_from=0, to=5):
    fig = go.Figure()
    for i in self.price_dic:
      if (i in self.result_df['종목코드'][start_from:to].values):
        fig.add_trace(go.Scatter(x= self.price_dic[i].index, \
                                 y= self.price_dic[i]['Close']/self.price_dic[i]['Close'][0], \
                                 name= self.filtered_df[self.filtered_df['종목코드']==i]['종목명'].item()))

    fig.update_layout(title=self.today,
                      xaxis_title='Date',
                      yaxis_title='Return (%)')
    fig.show()

  def plot_stock(self, id):
    t = stocklist.price_dic[id]
    candlesticks = go.Candlestick(
        x=t.index,
        open=t['Open'],
        high=t['High'],
        low=t['Low'],
        close=t['Close'],
        showlegend=False
    )

    volume_bars = go.Bar(
        x=t.index,
        y=t['Volume'],
        showlegend=False,
        marker={
            "color": "rgba(128,128,128,0.5)",
        }
    )

    fig = go.Figure(candlesticks)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(candlesticks, secondary_y=False
                  )
    fig.add_trace(volume_bars, secondary_y=True)
    fig.update_layout(title= self.filtered_df[self.filtered_df['종목코드']==id]['종목명'].item(), height=800)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
    fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
    fig.show()

In [37]:
highest = [0.99, 1]
higher = [0.9, 0.99]
longer = 500

In [48]:
stocklist = StockList()

In [52]:
t = stocklist.result_df
t

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
777,047050,포스코인터내셔널,24650,27950,3300,13.39,184774156,4738486136480,33.343329,73.156034
271,042660,대우조선해양,21700,28100,6400,29.49,104157193,2304274885080,30.872791,42.984207
1172,078600,대주전자재료,87000,107900,20900,24.02,43970768,4358737143400,30.398733,48.321337
334,000150,두산,83100,97800,14700,17.69,31287429,2799864000100,27.815046,45.822222
2032,049070,인탑스,31850,37400,5550,17.43,95110766,3317422162650,25.848386,48.892041
2381,067310,하나마이크론,12750,15960,3210,25.18,202786304,2773491997490,25.620175,52.362582
855,052690,한전기술,72300,80000,7700,10.65,62512707,4121155146900,25.327575,50.275522
634,007660,이수페타시스,7220,9200,1980,27.42,561362818,4891266004245,25.232957,55.912442
931,005010,휴스틸,5316,6480,1164,21.90,317529809,2078946556350,24.889188,36.438793
1231,376300,디어유,36850,41000,4150,11.26,70862308,2954391811690,23.723014,44.958153


In [53]:
t = t[['종목명', '종료일 종가']].reset_index(drop=True).rename({'종료일 종가':'현재가'}, axis=1)
t.index+=1
t.index.name='순위'
t

,종목명,현재가
순위,,
1,포스코인터내셔널,27950
2,대우조선해양,28100
3,대주전자재료,107900
4,두산,97800
5,인탑스,37400
6,하나마이크론,15960
7,한전기술,80000
8,이수페타시스,9200
9,휴스틸,6480


In [54]:
from google.colab import files
t.to_csv('df.csv', encoding="utf-8-sig") 
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>